In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook2").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1g").\
        config("spark.mongodb.input.uri","mongodb://mongo1:27017,mongo2:27018,mongo3:27019/database.horses_collection?replicaSet=rs0").\
        config("spark.mongodb.output.uri","mongodb://mongo1:27017,mongo2:27018,mongo3:27019/database.horses_collection?replicaSet=rs0").\
        config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").\
        getOrCreate()

In [89]:
from pyspark.sql.functions import explode

In [90]:
#reading dataframes from MongoDB
df = spark.read.format("mongo").load()

In [91]:
df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- clk: string (nullable = true)
 |-- mc: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- marketDefinition: struct (nullable = true)
 |    |    |    |-- betDelay: integer (nullable = true)
 |    |    |    |-- bettingType: string (nullable = true)
 |    |    |    |-- bspMarket: boolean (nullable = true)
 |    |    |    |-- bspReconciled: boolean (nullable = true)
 |    |    |    |-- complete: boolean (nullable = true)
 |    |    |    |-- countryCode: string (nullable = true)
 |    |    |    |-- crossMatching: boolean (nullable = true)
 |    |    |    |-- discountAllowed: boolean (nullable = true)
 |    |    |    |-- eachWayDivisor: double (nullable = true)
 |    |    |    |-- eventId: string (nullable = true)
 |    |    |    |-- eventName: string (nullable = true)
 |    |    |    |-- eventTypeId: string (nullable = true)
 | 

In [92]:
mc_exploded = df.select('*', explode(df.mc).alias('mc_row'))

In [93]:
mc_exploded.limit(10).show()

+--------------------+----------+--------------------+---+-------------+--------------------+
|                 _id|       clk|                  mc| op|           pt|              mc_row|
+--------------------+----------+--------------------+---+-------------+--------------------+
|[619e9dd8677066b2...|2480088678|[[1.124699287, [0...|mcm|1462537993283|[1.124699287, [0,...|
|[619e9dd8677066b2...|2480249142|[[1.124699287, [0...|mcm|1462540135101|[1.124699287, [0,...|
|[619e9dd8677066b2...|2481803835|[[1.124699287, [0...|mcm|1462555384075|[1.124699287, [0,...|
|[619e9dd8677066b2...|2481994285|[[1.124699287,, [...|mcm|1462557371191|[1.124699287,, [[...|
|[619e9dd8677066b2...|2482055516|[[1.124699287,, [...|mcm|1462557910955|[1.124699287,, [[...|
|[619e9dd8677066b2...|2482072939|[[1.124699287,, [...|mcm|1462558089056|[1.124699287,, [[...|
|[619e9dd8677066b2...|2482078964|[[1.124699287,, [...|mcm|1462558150717|[1.124699287,, [[...|
|[619e9dd8677066b2...|2482083965|[[1.124699287, [0...|mcm|14

In [28]:
#mc_exploded = mc_exploded.limit(1000)

In [29]:
#md_only = mc_exploded.filter(mc_exploded.mc_row.marketDefinition.isNotNull())

In [94]:
#md_only = md_only.where(market_definitions.marketType == "WIN") #.collect()

md_only = mc_exploded.filter(mc_exploded.mc_row.marketDefinition.isNotNull()) ##and mc_exploded.where(mc_exploded.mc_row.marketDefinition.marketType == "WIN") and mc_exploded.where(mc_exploded.mc_row.marketDefinition.countryCode == "GB") and mc_exploded.where(mc_exploded.mc_row.id == "1.124691542")

In [50]:
##md_only = mc_exploded.filter(mc_exploded.mc_row.marketDefinition.isNotNull()) and mc_exploded.where(mc_exploded.mc_row.id == "1.124691542")

In [95]:
# Explode the marketDefinition array.
market_definitions = md_only.selectExpr('op AS operation_type',
                                        'clk AS sequence_token',
                                        'pt AS published_time',
                                        'mc_row.id AS market_id',
                                        'mc_row.rc AS rc',
                                        'mc_row.marketDefinition.betDelay AS bet_delay',
                                        'mc_row.marketDefinition.bettingType AS betting_type',
                                        'mc_row.marketDefinition.bspMarket AS bsp_market',
                                        'mc_row.marketDefinition.bspReconciled AS bsp_reconciled',
                                        'mc_row.marketDefinition.complete AS complete',
                                        'mc_row.marketDefinition.countryCode AS country_code',
                                        'mc_row.marketDefinition.crossMatching AS cross_matching',
                                        'mc_row.marketDefinition.discountAllowed AS discount_allowed',
                                        'mc_row.marketDefinition.eventId AS event_id',
                                        'mc_row.marketDefinition.eventName AS event_name',
                                        'mc_row.marketDefinition.eventTypeId AS event_type_id',
                                        'mc_row.marketDefinition.inPlay AS in_play',
                                        'mc_row.marketDefinition.marketBaseRate AS market_base_rate',
                                        'mc_row.marketDefinition.marketTime AS market_time',
                                        'mc_row.marketDefinition.marketType AS market_type',
                                        'mc_row.marketDefinition.numberOfActiveRunners AS number_of_active_runners',
                                        'mc_row.marketDefinition.numberOfWinners AS number_of_winners',
                                        'mc_row.marketDefinition.openDate AS open_date',
                                        'mc_row.marketDefinition.persistenceEnabled AS persistence_enabled',
                                        'mc_row.marketDefinition.runnersVoidable AS runners_voidable',
                                        'mc_row.marketDefinition.settledTime AS settled_time',
                                        'mc_row.marketDefinition.status AS status',
                                        'mc_row.marketDefinition.suspendTime AS suspend_time',
                                        'mc_row.marketDefinition.timezone AS timezone',
                                        'mc_row.marketDefinition.turnInPlayEnabled AS turn_in_play_enabled',
                                        'mc_row.marketDefinition.version AS version',
                                        'mc_row.marketDefinition.name AS market_name',
                                        'mc_row.marketDefinition.regulators AS regulators',
                                        'mc_row.marketDefinition.runners AS runners')

In [96]:
market_definitions = market_definitions.filter(market_definitions["status"]=="CLOSED")

#market_definitions = market_definitions.where(market_definitions.country_code == "GB") #.collect()

#market_definitions = market_definitions.where(market_definitions.market_type == "WIN") #.collect()

In [97]:
#market_definitions = market_definitions.where(market_definitions.market_id == "1.124664295") #.collect()

market_definitions = market_definitions.filter(market_definitions["market_type"]=="WIN")

AnalysisException: cannot resolve '(`runners`.`status` = 'REMOVED')' due to data type mismatch: differing types in '(`runners`.`status` = 'REMOVED')' (array<string> and string).;;
'Filter NOT (runners#1212.status = REMOVED)
+- Filter (market_type#1198 = WIN)
   +- Filter (status#1205 = CLOSED)
      +- Project [op#1134 AS operation_type#1179, clk#1132 AS sequence_token#1180, pt#1135L AS published_time#1181L, mc_row#1142.id AS market_id#1182, mc_row#1142.rc AS rc#1183, mc_row#1142.marketDefinition.betDelay AS bet_delay#1184, mc_row#1142.marketDefinition.bettingType AS betting_type#1185, mc_row#1142.marketDefinition.bspMarket AS bsp_market#1186, mc_row#1142.marketDefinition.bspReconciled AS bsp_reconciled#1187, mc_row#1142.marketDefinition.complete AS complete#1188, mc_row#1142.marketDefinition.countryCode AS country_code#1189, mc_row#1142.marketDefinition.crossMatching AS cross_matching#1190, mc_row#1142.marketDefinition.discountAllowed AS discount_allowed#1191, mc_row#1142.marketDefini

In [98]:
#market_definitions = market_definitions.limit(1000)

In [99]:
market_definitions.head()

Row(operation_type='mcm', sequence_token='2478655026', published_time=1462501034557, market_id='1.124659130', rc=None, bet_delay=0, betting_type='ODDS', bsp_market=True, bsp_reconciled=True, complete=True, country_code='US', cross_matching=False, discount_allowed=True, event_id='27784588', event_name='EvangD (US) 5th May', event_type_id='7', in_play=False, market_base_rate=5.0, market_time='2016-05-06T01:59:00.000Z', market_type='WIN', number_of_active_runners=0, number_of_winners=1, open_date='2016-05-05T22:50:00.000Z', persistence_enabled=True, runners_voidable=False, settled_time='2016-05-06T02:16:36.000Z', status='CLOSED', suspend_time='2016-05-06T01:59:00.000Z', timezone='US/Central', turn_in_play_enabled=False, version=1308165702, market_name='R8 1m Allw', regulators=['MR_NJ', 'MR_INT'], runners=[Row(adjustmentFactor=1.12, bsp=None, id=10861746, name='B Brazin', removalDate='2016-05-05T15:33:44.000Z', sortPriority=1, status='REMOVED'), Row(adjustmentFactor=11.05, bsp=None, id=451

In [112]:
# Filter down to only the records that have Runners.
runners_only = runners_only.filter(runners_only.runners.status =="WINNER" ) 


#runners_only = runners_only.where(runners_only.status == "WINNER") 

#where(mc_exploded.mc_row.marketDefinition.countryCode == "GB") 
#market_definitions.filter(market_definitions["market_type"]=="WIN")

AnalysisException: cannot resolve '(`runners`.`status` = 'WINNER')' due to data type mismatch: differing types in '(`runners`.`status` = 'WINNER')' (array<string> and string).;;
'Filter (runners#1212.status = WINNER)
+- Filter isnotnull(runners#1212)
   +- Filter (market_type#1198 = WIN)
      +- Filter (status#1205 = CLOSED)
         +- Project [op#1134 AS operation_type#1179, clk#1132 AS sequence_token#1180, pt#1135L AS published_time#1181L, mc_row#1142.id AS market_id#1182, mc_row#1142.rc AS rc#1183, mc_row#1142.marketDefinition.betDelay AS bet_delay#1184, mc_row#1142.marketDefinition.bettingType AS betting_type#1185, mc_row#1142.marketDefinition.bspMarket AS bsp_market#1186, mc_row#1142.marketDefinition.bspReconciled AS bsp_reconciled#1187, mc_row#1142.marketDefinition.complete AS complete#1188, mc_row#1142.marketDefinition.countryCode AS country_code#1189, mc_row#1142.marketDefinition.crossMatching AS cross_matching#1190, mc_row#1142.marketDefinition.discountAllowed AS discount_al

In [101]:
runners_only.head(1)

[Row(operation_type='mcm', sequence_token='2478655026', published_time=1462501034557, market_id='1.124659130', rc=None, bet_delay=0, betting_type='ODDS', bsp_market=True, bsp_reconciled=True, complete=True, country_code='US', cross_matching=False, discount_allowed=True, event_id='27784588', event_name='EvangD (US) 5th May', event_type_id='7', in_play=False, market_base_rate=5.0, market_time='2016-05-06T01:59:00.000Z', market_type='WIN', number_of_active_runners=0, number_of_winners=1, open_date='2016-05-05T22:50:00.000Z', persistence_enabled=True, runners_voidable=False, settled_time='2016-05-06T02:16:36.000Z', status='CLOSED', suspend_time='2016-05-06T01:59:00.000Z', timezone='US/Central', turn_in_play_enabled=False, version=1308165702, market_name='R8 1m Allw', regulators=['MR_NJ', 'MR_INT'], runners=[Row(adjustmentFactor=1.12, bsp=None, id=10861746, name='B Brazin', removalDate='2016-05-05T15:33:44.000Z', sortPriority=1, status='REMOVED'), Row(adjustmentFactor=11.05, bsp=None, id=45

In [ ]:
runners_only = market_definitions.filter(market_definitions.runners.isNotNull()) 

In [105]:
# Explode the Runners array.
runners_exploded = runners_only.select(market_definitions.operation_type,
                                       market_definitions.published_time,
                                       market_definitions.market_id,
                                       market_definitions.market_type,
                                       market_definitions.market_name,
                                       market_definitions.event_id,
                                       market_definitions.event_name,
                                       explode(market_definitions.runners).alias('runner_row'))

In [107]:
runners_exploded.head(5)

[Row(operation_type='mcm', published_time=1462501034557, market_id='1.124659130', market_type='WIN', market_name='R8 1m Allw', event_id='27784588', event_name='EvangD (US) 5th May', runner_row=Row(adjustmentFactor=1.12, bsp=None, id=10861746, name='B Brazin', removalDate='2016-05-05T15:33:44.000Z', sortPriority=1, status='REMOVED')),
 Row(operation_type='mcm', published_time=1462501034557, market_id='1.124659130', market_type='WIN', market_name='R8 1m Allw', event_id='27784588', event_name='EvangD (US) 5th May', runner_row=Row(adjustmentFactor=11.05, bsp=None, id=4517788, name='Great Value', removalDate='2016-05-05T15:33:56.000Z', sortPriority=2, status='REMOVED')),
 Row(operation_type='mcm', published_time=1462501034557, market_id='1.124659130', market_type='WIN', market_name='R8 1m Allw', event_id='27784588', event_name='EvangD (US) 5th May', runner_row=Row(adjustmentFactor=5.06, bsp=None, id=187960, name='Kid Creole', removalDate='2016-05-05T15:34:05.000Z', sortPriority=3, status='R

In [85]:
# Select the useful fields, and give them user friendly names.
runners = runners_exploded.selectExpr('operation_type',
                                      'published_time',
                                      'market_id',
                                      'market_type',
                                      'market_name',
                                      'event_id',
                                      'event_name',
                                      'runner_row.id AS runner_id',
                                      'runner_row.name AS runner_name',
                                      'runner_row.status AS runner_status',
                                      'runner_row.sortPriority AS runner_sort_priority')

In [86]:
runners.head(1)

[Row(operation_type='mcm', published_time=1462501034557, market_id='1.124659130', market_type='WIN', market_name='R8 1m Allw', event_id='27784588', event_name='EvangD (US) 5th May', runner_id=10861746, runner_name='B Brazin', runner_status='REMOVED', runner_sort_priority=1)]

In [87]:
runners.show()

+--------------+--------------+-----------+-----------+---------------+--------+-------------------+---------+------------------+-------------+--------------------+
|operation_type|published_time|  market_id|market_type|    market_name|event_id|         event_name|runner_id|       runner_name|runner_status|runner_sort_priority|
+--------------+--------------+-----------+-----------+---------------+--------+-------------------+---------+------------------+-------------+--------------------+
|           mcm| 1462501034557|1.124659130|        WIN|     R8 1m Allw|27784588|EvangD (US) 5th May| 10861746|          B Brazin|      REMOVED|                   1|
|           mcm| 1462501034557|1.124659130|        WIN|     R8 1m Allw|27784588|EvangD (US) 5th May|  4517788|       Great Value|      REMOVED|                   2|
|           mcm| 1462501034557|1.124659130|        WIN|     R8 1m Allw|27784588|EvangD (US) 5th May|   187960|        Kid Creole|      REMOVED|                   3|
|         

In [62]:
# Filter down to only the records that have rc (runner changes).
##rc_only = mc_exploded.filter(mc_exploded.mc_row.rc.isNotNull()) and mc_exploded.where(mc_exploded.mc_row.id == "1.124691542") 

In [63]:
#rc_only.head(1)

[Row(_id=Row(oid='619e9dd8677066b2230c1e02'), clk='2461683065', mc=[Row(id='1.124691542', marketDefinition=Row(betDelay=0, bettingType='ODDS', bspMarket=False, bspReconciled=False, complete=True, countryCode='GB', crossMatching=False, discountAllowed=True, eachWayDivisor=None, eventId='27786741', eventName='Ling (RFC) 6th May', eventTypeId='7', inPlay=False, marketBaseRate=5.0, marketTime='2016-05-06T12:30:00.000Z', marketType='REV_FORECAST', name='Reverse FC', numberOfActiveRunners=36, numberOfWinners=1, openDate='2016-05-06T12:30:00.000Z', persistenceEnabled=True, regulators=['MR_INT'], runners=[Row(adjustmentFactor=None, bsp=None, id=2521889, name='1 - 2 / 2 - 1', removalDate=None, sortPriority=1, status='ACTIVE'), Row(adjustmentFactor=None, bsp=None, id=2521890, name='1 - 3 / 3 - 1', removalDate=None, sortPriority=2, status='ACTIVE'), Row(adjustmentFactor=None, bsp=None, id=2521892, name='1 - 4 / 4 - 1', removalDate=None, sortPriority=3, status='ACTIVE'), Row(adjustmentFactor=None,

In [64]:
# Explode the rc (runner changes) array.
#rc_exploded = rc_only.select(rc_only.op.alias('operation_type'),
 #                            rc_only.pt.alias('published_time'),
  #                           rc_only.mc_row.id.alias('market_id'),
   #                          explode(rc_only.mc_row.rc).alias('runner_change_row'))

In [65]:
rc_exploded.head(1)

Py4JJavaError: An error occurred while calling o423.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 30.0 failed 4 times, most recent failure: Lost task 0.3 in stage 30.0 (TID 289, 172.20.0.7, executor 0): org.apache.spark.sql.catalyst.errors.package$TreeNodeException: Binding attribute, tree: _gen_alias_828#828
	at org.apache.spark.sql.catalyst.errors.package$.attachTree(package.scala:56)
	at org.apache.spark.sql.catalyst.expressions.BindReferences$$anonfun$bindReference$1.applyOrElse(BoundAttribute.scala:75)
	at org.apache.spark.sql.catalyst.expressions.BindReferences$$anonfun$bindReference$1.applyOrElse(BoundAttribute.scala:74)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$transformDown$1(TreeNode.scala:309)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:72)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformDown(TreeNode.scala:309)
	at org.apache.spark.sql.catalyst.trees.TreeNo

In [66]:
# Select the useful fields, and give them user friendly names.
runner_changes = rc_exploded.selectExpr('operation_type',
                                        'published_time',
                                        'market_id',
                                        'runner_change_row.id AS runner_id',
                                        'runner_change_row.ltp AS last_traded_price')

In [67]:
runner_changes.head()

Py4JJavaError: An error occurred while calling o436.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 31.0 failed 4 times, most recent failure: Lost task 0.3 in stage 31.0 (TID 293, 172.20.0.8, executor 1): org.apache.spark.sql.catalyst.errors.package$TreeNodeException: Binding attribute, tree: _gen_alias_843#843
	at org.apache.spark.sql.catalyst.errors.package$.attachTree(package.scala:56)
	at org.apache.spark.sql.catalyst.expressions.BindReferences$$anonfun$bindReference$1.applyOrElse(BoundAttribute.scala:75)
	at org.apache.spark.sql.catalyst.expressions.BindReferences$$anonfun$bindReference$1.applyOrElse(BoundAttribute.scala:74)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$transformDown$1(TreeNode.scala:309)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:72)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformDown(TreeNode.scala:309)
	at org.apache.spark.sql.catalyst.trees.TreeNo

In [68]:
# one market_id 
selection_condition = "market_id = '1.124691542'" # "market_id = '1.124664295'"

chosen_runners = runners.filter(selection_condition).selectExpr('market_id', 
                                                                'event_name', 
                                                                'market_name', 
                                                                'runner_id', 
                                                                'runner_name').distinct()


In [69]:

# Runners have a 1:many relationship to Runner Changes, so do an inner join from Runners to Runner Changes.
chosen_runner_changes = chosen_runners.join(runner_changes, chosen_runners.runner_id == runner_changes.runner_id).select(
                        chosen_runners.market_id, 
                        chosen_runners.event_name, 
                        chosen_runners.market_name, 
                        chosen_runners.runner_id, 
                        chosen_runners.runner_name,
                        runner_changes.published_time,
                        runner_changes.last_traded_price
                        ).orderBy('published_time').collect()

KeyboardInterrupt: 

In [ ]:
# Betfair time seems to be in milliseconds, so this converts it to normal Unix epoch time in seconds.
def betfair_to_epoch(betfair_time):
    return betfair_time / 1000

# Convert a string like '30.08.2018 17:30:00' to a time number.
def string_to_epoch(time_str):
    pattern = '%d.%m.%Y %H:%M:%S'
    return time.mktime(time.strptime(time_str, pattern))

In [3]:
#### stephen's code

In [4]:
df1 = df.select("_id")

In [7]:
df.limit(10).show()

+--------------------+----------+--------------------+---+-------------+
|                 _id|       clk|                  mc| op|           pt|
+--------------------+----------+--------------------+---+-------------+
|[619e9dd8677066b2...|2480088678|[[1.124699287, [0...|mcm|1462537993283|
|[619e9dd8677066b2...|2480249142|[[1.124699287, [0...|mcm|1462540135101|
|[619e9dd8677066b2...|2481803835|[[1.124699287, [0...|mcm|1462555384075|
|[619e9dd8677066b2...|2481994285|[[1.124699287,, [...|mcm|1462557371191|
|[619e9dd8677066b2...|2482055516|[[1.124699287,, [...|mcm|1462557910955|
|[619e9dd8677066b2...|2482072939|[[1.124699287,, [...|mcm|1462558089056|
|[619e9dd8677066b2...|2482078964|[[1.124699287,, [...|mcm|1462558150717|
|[619e9dd8677066b2...|2482083965|[[1.124699287, [0...|mcm|1462558199155|
|[619e9dd8677066b2...|2482177634|[[1.124699287, [0...|mcm|1462559078531|
|[619e9dd8677066b2...|2480088678|[[1.124699285, [0...|mcm|1462537993283|
+--------------------+----------+------------------

In [6]:
df1.count()

Py4JJavaError: An error occurred while calling o54.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 3.0 failed 4 times, most recent failure: Lost task 1.3 in stage 3.0 (TID 132, 172.20.0.8, executor 1): com.mongodb.spark.exceptions.MongoTypeConversionException: Cannot cast STRING into a DoubleType (value: BsonString{value='NaN'})
	at com.mongodb.spark.sql.MapFunctions$.convertToDataType(MapFunctions.scala:214)
	at com.mongodb.spark.sql.MapFunctions$.$anonfun$documentToRow$1(MapFunctions.scala:37)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:238)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at scala.collection.TraversableLike.map(TraversableLike.scala:238)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:231)
	at scal